## TEXT PREPROCESSING

This notebook is for preprocessing text (abstracts) from Kaggle dataset CORD-19. (Used for dataset version 2) 

In [1]:
import pandas as pd                        
import numpy as np                         
import seaborn as sns                      
import matplotlib.pyplot as plt            
import re                                 
import string                              
import warnings                            
import nltk                                
from nltk.corpus import stopwords         
from gensim.corpora import Dictionary      

In [2]:
warnings.filterwarnings('ignore')

### Load Kaggle dataset with abstracts + some filtering 

In [3]:
pd.read_csv("sources/metadata.csv")

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659497,1bx1f3vg,4efbaad70dad7d5fd30065e507aa0fcbf3a57118,Medline; PMC,Evaluation of the impact of the COVID-19 lockd...,10.1007/s00432-021-03621-7,PMC8021215,33821320.0,cc-by,PURPOSE: The benefits of regular physical exer...,2021-04-05,"Motton, Stéphanie; Vergriete, Kelig; VanPhi, L...",J Cancer Res Clin Oncol,NaN,NaN,NaN,document_parses/pdf_json/4efbaad70dad7d5fd3006...,document_parses/pmc_json/PMC8021215.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33821320/;...,233030613.0
659498,fffrnhbx,7e1c78e84a86c94c9da79297c2d0c9820af0e30f,Medline; PMC,SARS-CoV-2 infection fatality rate among elder...,10.1093/cid/ciaa1627,PMC7665387,33103182.0,cc-by-nc-nd,BACKGROUND: Despite the vast majority of indiv...,2020-10-26,"Pedersen, Ole Birger; Nissen, Janna; Dinh, Kho...",Clin Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/7e1c78e84a86c94c9da79...,NaN,https://www.ncbi.nlm.nih.gov/pubmed/33103182/;...,225069963.0
659499,crz52oo8,6b04f6082c05d936f11d395a5c70ce2edf37bf57,BioRxiv; WHO,De novo design of high-affinity antibody varia...,10.1101/2020.04.09.034868,NaN,NaN,biorxiv,The emergence of SARS-CoV-2 is responsible for...,2020-05-11,"Boorla, Veda Sheersh; Chowdhury, Ratul; Marana...",bioRxiv,NaN,NaN,NaN,document_parses/pdf_json/6b04f6082c05d936f11d3...,NaN,https://doi.org/10.1101/2020.04.09.034868,215790698.0
659500,1bxxmmc1,f4ead62f942cf3d4f4a963e4ee72e05db58da631,Elsevier; Medline; PMC,Biophysicists' outstanding response to Covid-19,10.1016/j.bpj.2021.02.020,PMC7931721,33689685.0,no-cc,NaN,2021-03-04,"Schlick, Tamar; Sundberg, Eric J.; Schroeder, ...",Biophys J,NaN,NaN,NaN,document_parses/pdf_json/f4ead62f942cf3d4f4a96...,document_parses/pmc_json/PMC7931721

In [5]:
df_all = pd.read_csv("sources/metadata.csv")
df_all['Year'] = pd.DatetimeIndex(df_all['publish_time']).year
df_all['Month'] = pd.DatetimeIndex(df_all['publish_time']).month
df_all = df_all[["Year","Month","abstract","doi","cord_uid"]]
print("Number of all articles before cleaning and filtering (are duplicated across sources): "+str(len(df_all)))

df_all = df_all.drop_duplicates("doi")
print("Number of articles of unique doi :" + str(len(df_all)))

df_all = df_all[pd.notnull(df_all['abstract'])]
print("Number of articles of unique doi with not null abstracts:" + str(len(df_all)))

df_all['len'] =df_all['abstract'].map(len)
df_all = df_all[df_all['len']>10]
print("Number of articles of unique doi with len of abstract > 10:" + str(len(df_all)))

Number of all articles before cleaning and filtering (are duplicated across sources): 659502
Number of articles of unique doi :332893
Number of articles of unique doi with not null abstracts:255902
Number of articles of unique doi with len of abstract > 10:255639


#  TExt cleaning 


- 1 313 622 words from the not cleaned abstracts of the 255 639 articles 
- Number of unique words after all that: 39 657
- Number of unique words after all that for articles with target can be different because articles will be reduced by target!

**Text cleaning steps**:
- Removed rare words (which are in less than 5 articles) 
- Removed some predefined stop words
- Removed punctions except / and – (which are often used to connect words) / and – replaced with „“
- Converted to lower case
- deleted single numbers (not of len = 4 – dont want to delete years)
- Lemmatization (convert the word into root word)

### Create functions for removing punctions, stopwords and lemmatization

In [4]:
def remove_punc_lower(text):   
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    # I dont want to remove dash, because dash is connection between words!
    remove = string.punctuation
    remove = remove.replace("-", "")     
    remove = remove.replace("/", "")
    pattern = r"[{}]".format(remove)
    text = re.sub(pattern, "", str(text).lower().strip())    
    return text

In [5]:
def lemm(text):
    lst_text = text.split()
    lem = nltk.stem.wordnet.WordNetLemmatizer()    
    lst_text = [lem.lemmatize(word) for word in lst_text]
    text = " ".join(lst_text)  
    return text

In [6]:
def delete_numbers(text):
    lst_text = text.split()
    lst_text = [word for word in lst_text if not (word.isdecimal()==True) & (len(word)!=4 ) ]
    text = " ".join(lst_text)
    return text

### Create list of removing words

In [7]:
%%time

corpus = df_all.abstract.values

dataset = [d.split() for d in corpus]
dct = Dictionary(dataset)
print("Number of all words is :" + str(len(dct) ))
print("from original abstracts of number of articles: " + str(len(df_all)))

dct.filter_extremes(no_below=5)                 
print("Number of all words after removing rare words (contained below 5 articles) :" + str(len(dct) ))

# dont remove negative from stopwords (it can have importance)
stop_words = stopwords.words('english')
remove_from_stopwords = ["against","don't","no","not","aren't","didn't","doesn't","needn't","isn't","wasn't","weren't","wouldn't"]
stop_words = [ x for x in stop_words if x is not remove_from_stopwords ]

stop_words.extend(["abstract","background","objective"])
    
list_keep_words = list(dct.itervalues())
list_keep_words = list(set(list_keep_words) ^ set(stop_words))
print("Then after removing stopwords: "+ str(len(list_keep_words)))

list_removed_words = list(set(list(Dictionary(dataset).itervalues())) ^ set(list_keep_words))
list_removed_words.extend(stop_words)

Number of all words is :1313622
from original abstracts of number of articles: 255639
Number of all words after removing rare words (contained below 5 articles) :100000
Then after removing stopwords: 99924
Wall time: 2min 51s


In [8]:
len(list_removed_words)

1213932

### Cleaning process

In [9]:
%%time
cache = set(list_removed_words)
df_all["abstract_cleaned"]  = df_all['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word not in cache]))

Wall time: 21.2 s


In [10]:
%%time
df_all["abstract_cleaned"] = df_all["abstract_cleaned"].astype(str).apply(lambda x: remove_punc_lower(x))

Wall time: 6.21 s


In [11]:
%%time
df_all["abstract_cleaned"] = df_all["abstract_cleaned"].astype(str).apply(lambda x: lemm(x))

Wall time: 3min 8s


In [12]:
%%time
df_all["abstract_cleaned"]  = df_all['abstract_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in cache]))

Wall time: 10.1 s


### Some examples for checking:

In [13]:
df_all["abstract_cleaned"] = df_all["abstract_cleaned"].astype(str).apply(lambda x: delete_numbers(x))
print(df_all[['abstract',"abstract_cleaned"]][1:2].values)

[['Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the l

In [14]:
print(df_all[['abstract',"abstract_cleaned"]][3:4].values)

[['Endothelin-1 (ET-1) is a 21 amino acid peptide with diverse biological activity that has been implicated in numerous diseases. ET-1 is a potent mitogen regulator of smooth muscle tone, and inflammatory mediator that may play a key role in diseases of the airways, pulmonary circulation, and inflammatory lung diseases, both acute and chronic. This review will focus on the biology of ET-1 and its role in lung disease.'
  'amino acid peptide diverse biological activity implicated numerous disease et-1 potent mitogen regulator smooth muscle tone inflammatory mediator may play key role disease airway pulmonary circulation inflammatory lung disease acute chronic review focus biology et-1 role lung disease']]


In [15]:
print(df_all[['abstract',"abstract_cleaned"]][9:10].values)

[['This report describes the design and implementation of the Real-time Outbreak and Disease Surveillance (RODS) system, a computer-based public health surveillance system for early detection of disease outbreaks. Hospitals send RODS data from clinical encounters over virtual private networks and leased lines using the Health Level 7 (HL7) message protocol. The data are sent in real time. RODS automatically classifies the registration chief complaint from the visit into one of seven syndrome categories using Bayesian classifiers. It stores the data in a relational database, aggregates the data for analysis using data warehousing techniques, applies univariate and multivariate statistical detection algorithms to the data, and alerts users of when the algorithms identify anomalous patterns in the syndrome counts. RODS also has a Web-based user interface that supports temporal and spatial analyses. RODS processes sales of over-the-counter health care products in a similar manner but recei

In [16]:
print(df_all[['abstract',"abstract_cleaned"]][21:22].values)

[['Horizontal DNA transfer is an important factor of evolution and participates in biological diversity. Unfortunately, the location and length of horizontal transfers (HTs) are known for very few species. The usage of short oligonucleotides in a sequence (the so-called genomic signature) has been shown to be species-specific even in DNA fragments as short as 1 kb. The genomic signature is therefore proposed as a tool to detect HTs. Since DNA transfers originate from species with a signature different from those of the recipient species, the analysis of local variations of signature along recipient genome may allow for detecting exogenous DNA. The strategy consists in (i) scanning the genome with a sliding window, and calculating the corresponding local signature (ii) evaluating its deviation from the signature of the whole genome and (iii) looking for similar signatures in a database of genomic signatures. A total of 22 prokaryote genomes are analyzed in this way. It has been observed

In [17]:
corpus = df_all.abstract_cleaned.values
dataset = [d.split() for d in corpus]
dct = Dictionary(dataset)
print("Number of words after cleaning: "+str(len(dct)))

Number of words after cleaning: 39653


# FInal export

In [19]:
df_all['len'] =df_all['abstract_cleaned'].astype(str).map(len)
df_final_cleaned = df_all[df_all['len']>5]

In [20]:
print("Number of articles of cleaned abstracts with len of abstract > 5:" + str(len(df_final_cleaned)))

Number of articles of cleaned abstracts with len of abstract > 5:255500


In [3]:
df_final_cleaned.to_csv("1.Preprocessing_outputs/df_sw_tok_low_punc_lemm_v5.csv")

NameError: name 'df_final_cleaned' is not defined

In [8]:
df_all = pd.read_csv("1.Preprocessing_outputs/df_sw_tok_low_punc_lemm_v5.csv")

In [9]:
df_all

,Unnamed: 0,Year,Month,abstract,doi,cord_uid,len,abstract_cleaned
0,0,2001.0,7.0,OBJECTIVE: This retrospective chart review des...,10.1186/1471-2334-1-6,ug7v899j,1061,retrospective chart review describes epidemiol...
1,1,2000.0,8.0,Inflammatory diseases of the respiratory tract...,10.1186/rr14,02tnwd4m,686,inflammatory disease respiratory tract commonl...
2,2,2000.0,8.0,Surfactant protein-D (SP-D) participates in th...,10.1186/rr19,ejv2xln0,1193,surfactant sp-d participates innate response i...
3,3,2001.0,2.0,Endothelin-1 (ET-1) is a 21 amino acid peptide...,10.1186/rr44,2b73a28n,284,amino acid peptide diverse biological activity...
4,4,2001.0,5.0,Respiratory syncytial virus (RSV) and pneumoni...,10.1186/rr61,9785vg6d,617,respiratory syncytial virus rsv pneumonia viru...
...,...,...,...,...,...,...,...,...
255495,659496,2020.0,4.0,Public Health professionals and academics have...,10.23750/abm.v91i3-s.9507,9havlnub,1259,public health professional academic frontline ...
255496,659497,2021.0,4.0,PURPOSE: The benefits of regular physical exer...,10.1007/s00432-021-03621-7,1bx1f3vg,1566,purpose benefit regular physical exercise tole...
255497,659498,2020.0,10.0,BACKGROUND: Despite the vast majority of indiv...,10.1093/cid/ciaa1627,fffrnhbx,1044,despite vast majority individual succumbing in...
255498,659499,2020.0,5.0,The emergence of SARS-CoV-2 is responsible for...,10.1101/2020.04.09.034868,crz52oo8,1181,emergence responsible pandemic respiratory dis...
